In [1]:
import numpy as np
import scipy as sp
import sympy as sm
import matplotlib.pyplot as plt

from scipy.optimize import linprog
from functools import partial

In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
    def __round__(self, d):
        new = self.copy()
        for i,v in new.items():
            try:
                new[i] = round(v,d)
                continue
            except:
                pass
            try:
                new[i] = v.round(d)
                continue
            except:
                pass
        return new

# 6-Decision. Lab2
## Michael Koreshkov. FI-91
> variant 6

In [3]:
PROCESSES = T1, T2, T3, T4, T5 = 'T1', 'T2', 'T3', 'T4', 'T5'
RESOURCES = R1, R2, R3, R4 = 'Raw(T)', 'KWh', 'Overhead(₴)', 'Salary(₴)'

M = len(PROCESSES)
N = len(RESOURCES)

In [4]:
# efficiency of processes
EFF = np.array([300, 260, 320, 400, 450])

# costs
COST = np.array([
    [15 , 20 , 15  , 14  , 18 ],
    [0.2, 0.3, 0.15, 0.25, 0.3],
    [4  , 5  , 6   , 3   , 2  ],
    [6  , 3  , 4   , 6   , 3   ]
])

# limit on the resources
LIM = np.array([2000, 300, 1000, 1600])

# criterion weights
CRITW = np.array([1,1])
CRITW = CRITW / CRITW.sum()

In [5]:
RESULTS = dotdict()

## 1. Math model

Нехай $X$ - вектор тривалостей роботи кожного процесу (в одиницях часу). Тоді

$$\begin{cases}
f_1(X) = EFF \cdot X \to max, \\
f_2(X) = COST_{2i} X_i \to min, \\
X \ge 0, \\
COST \cdot X \le LIM, \\
\end{cases}$$

Де $COST_{2i}$ відповідає `COST[1,i]`, тобто витратам електроенергії

## 2. Метод згортки


In [6]:
# 1) Normalize f1
f1 = dotdict()

res = linprog(-EFF, COST, LIM)
f1.max = -res.fun
f1.min = 0 # trivial

f1.A = -EFF / (f1.max - f1.min)
f1.b = f1.max / (f1.max - f1.min)

print(res)

     con: array([], dtype=float64)
     fun: -57142.85713243238
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([3.57873887e-07, 2.64285714e+02, 5.71428572e+02, 7.42857143e+02])
  status: 0
 success: True
       x: array([3.93557950e-10, 2.08057674e-10, 6.47707401e-10, 1.42857143e+02,
       2.19057949e-10])


In [7]:
# 2) Normalize f2
f2 = dotdict()

res = linprog(-COST[2,:], COST, LIM)
f2.max = -res.fun
f2.min = 0 # trivial

f2.A = COST[1,:] / (f2.max - f2.min)
f2.b = f2.min / (f2.max - f2.min)

print(res)

     con: array([], dtype=float64)
     fun: -799.9999701230996
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([7.37914404e-05, 2.80000001e+02, 2.00000030e+02, 1.06666669e+03])
  status: 0
 success: True
       x: array([1.06418233e-07, 2.85089885e-09, 1.33333328e+02, 1.93148149e-08,
       1.70608798e-08])


In [8]:
# 3) Linear convolution criterion

f = dotdict()
f.A = CRITW @ np.array([f1.A, f2.A])
f.b = CRITW @ np.array([f1.b, f2.b])

In [9]:
# 4) Optimization

optres = linprog(f.A, COST, LIM)

res = dotdict()
res.x = optres.x
res.profit = EFF @ optres.x
res.kwh = COST[1,:] @ optres.x

RESULTS.conv = res
print("Метод згортки з нормуванням. Результат")
res

Метод згортки з нормуванням. Результат


{'x': array([2.80908755e-09, 9.16858931e-09, 1.25236547e-08, 1.42857142e+02,
        4.37095089e-07]),
 'profit': 57142.85705417236,
 'kwh': 35.71428566772286}

### А тепер без нормування

In [10]:
optres = linprog(-EFF + COST[1,:], COST, LIM)

res = dotdict()
res.x = optres.x
res.profit = EFF @ optres.x
res.kwh = COST[1,:] @ optres.x

RESULTS.conv_nonorm = res
print("Метод згортки без нормування. Результат")
res

Метод згортки без нормування. Результат


{'x': array([3.97324607e-10, 2.09578141e-10, 6.55097202e-10, 1.42857143e+02,
        2.20689869e-10]),
 'profit': 57142.85713235626,
 'kwh': 35.71428570772783}

In [11]:
# різниця з випадком нормування
RESULTS.conv.x - res.x, np.linalg.norm(RESULTS.conv.x - res.x)

(array([ 2.41176294e-09,  8.95901117e-09,  1.18685575e-08, -7.04070487e-07,
         4.36874399e-07]),
 8.287348398877614e-07)

> Хочу звернути увагу, що різниця незначна 

## 3. Метод послідовної поступки

In [12]:
# concessions
CONCS = np.array([7000, 3])

In [13]:
# 1) f1
optres = linprog(-EFF, COST, LIM)

f1 = dotdict()
f1.max = -optres.fun

# 2) f2
# a new condition: f1 >= f1.max - CONCS[0]
A = np.vstack([COST, -EFF])
b = np.append(LIM, - f1.max + CONCS[0])

optres = linprog(COST[1,:], A, b)

f2 = dotdict()
f2.min = optres.fun

# 3) final optimization
# a new condition: f2 <= f1.min + CONCS[0]
A = np.vstack([A, COST[1,:]])
b = np.append(b, f2.min + CONCS[1])

optres = linprog(-EFF, A, b)

print(f1.max)
print(f2.min)

57142.85713243238
28.11560189753127


In [14]:
res = dotdict()
res.x = optres.x
res.profit = -optres.fun
res.kwh = COST[1,:] @ optres.x

RESULTS.concession = res
print("Метод послідовної поступки. Результат")
res

Метод послідовної поступки. Результат


{'x': array([1.83713595e-10, 5.60033432e-11, 3.90191354e+01, 1.01050926e+02,
        1.09580540e-10]),
 'profit': 52906.49386777832,
 'kwh': 31.115601896700724}

## 4. Метод головного критерію

In [15]:
# нехай головний критерій - f2
# знайдемо можливі значення f1

f1 = dotdict()

optres = linprog(-EFF, COST, LIM)
f1.max = -optres.fun
print("f1 max = ", f1.max)

# замінюємо критерій f1 на умову f1 >= 50000 (-f1 <= -50000)
A = np.vstack([COST, -EFF])
b = np.append(LIM, -50000)

optres = linprog(COST[1,:], A, b)

res = dotdict()
res.x = optres.x
res.profit = EFF @ res.x
res.kwh = optres.fun

RESULTS.maincrit = res
res

f1 max =  57142.85713243238


{'x': array([5.11111166e-11, 2.39382619e-11, 6.57894737e+01, 7.23684211e+01,
        5.05877909e-10]),
 'profit': 50000.00000007433,
 'kwh': 27.960526315923374}

Бачимо, що в даному випадку обмеження в 50000 профіту є вирішальним. Витрати електроенергії можна зменшувати до 0 і саме обмеження профіту визначає нижню межу КВг

In [16]:
# замінюємо критерій f1 на умову f1 >= 30000 (-f1 <= -30000)
A = np.vstack([COST, -EFF])
b = np.append(LIM, -30000)

optres = linprog(COST[1,:], A, b)

res = dotdict()
res.x = optres.x
res.profit = EFF @ res.x
res.kwh = optres.fun

res

{'x': array([1.32796099e-09, 1.29541457e-09, 9.37500000e+01, 3.95380697e-10,
        7.13546763e-10]),
 'profit': 30000.00001064343,
 'kwh': 14.062500005386962}

## Результати методв

In [17]:
R = dotdict()
for i,v in RESULTS.items():
    R[i] = round(v,3)
    
def rprint(res):
    print("X:", res['x'])
    print("profit:",res['profit'])
    print("kwh:",res['kwh'])

In [18]:
print("Згортка:")
rprint(R.conv)
print("\nЗгортка без нормування:")
rprint(R.conv_nonorm)
print("\nПослідовні поступки:")
rprint(R.concession)
print("\nГоловний критерій:")
rprint(R.maincrit)

Згортка:
X: [  0.      0.      0.    142.857   0.   ]
profit: 57142.857
kwh: 35.714

Згортка без нормування:
X: [  0.      0.      0.    142.857   0.   ]
profit: 57142.857
kwh: 35.714

Послідовні поступки:
X: [  0.      0.     39.019 101.051   0.   ]
profit: 52906.494
kwh: 31.116

Головний критерій:
X: [ 0.     0.    65.789 72.368  0.   ]
profit: 50000.0
kwh: 27.961
